In [1]:
import gurobipy as gp
import numpy as np
import pandas as pd

In [2]:
## Primal Problem Statement

primal = gp.Model("SOCP_Primal")

# Add Coefficients

P_objcoe = [2, 1, 1, -3]

P_INEQ_RHS1 = 7
P_INEQ_COE1 = [1, 0, 0, 0]

P_EQ_RHS = 4
P_EQ_COE = [2, -1, 1, 0.5]

P_INEQ_RHS2 = 15
P_INEQ_COE2 = [1, 1, 0, -2]

# Add Variables
x = {}

for i in range(2):
    x[i] = primal.addVar(name = 'x('+str(i)+')', lb = 0, ub = gp.GRB.INFINITY)
    
for i in range(2,4):
    x[i] = primal.addVar(name = 'x('+str(i)+')', lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY)
    
# Set Objective

primal.setObjective(sum(P_objcoe[i] * x[i] for i in range(4)), gp.GRB.MINIMIZE)

# Add Constraints

primal.addConstr(sum(P_INEQ_COE1[i] * x[i] for i in range(4)) <= P_INEQ_RHS1, 'INEQ1')
                 
primal.addConstr(sum(P_INEQ_COE2[i] * x[i] for i in range(4)) <= P_INEQ_RHS2, 'INEQ2')
                 
primal.addConstr(sum(P_EQ_COE[i] * x[i] for i in range(4)) == P_EQ_RHS, 'EQ')
                 
primal.addConstr(2 * x[0] * x[1] >= x[2] * x[2] + x[3] * x[3], 'CONE')

primal.update()

primal.setParam(gp.GRB.Param.Threads, 8)
primal.optimize()

Academic license - for non-commercial use only
Changed value of parameter Threads to 8
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 3 rows, 4 columns and 8 nonzeros
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [5e-01, 2e+00]
  QMatrix range    [1e+00, 2e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 4 rows, 6 columns, 13 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.20582557e+01 -9.82320422e+00  3.99e+00 3.63e+00  1.68e+01     0s
   1  -6.50953347e+00 -7.34399190e+00  8.16e-04 8.47e-01  2.19e+00     

In [2]:
## Primal Problem Statement

dual = gp.Model("SOCP_Dual")

# Add Coefficients

Q_objcoe_Y = [-7, 4, -15]
Q_objcoe_Z = [0, 0, 0, 0]

Q_INEQ_RHS1 = -2
Q_INEQ_COE1_Y = [1, -2, 1]
Q_INEQ_COE1_Z = [-1, 0, 0, 0]

Q_INEQ_RHS2 = -1
Q_INEQ_COE2_Y = [0, 1, 1]
Q_INEQ_COE2_Z = [0, -1, 0, 0]

Q_EQ_RHS1 = -1
Q_EQ_COE1_Y = [0, -1, 0]
Q_EQ_COE1_Z = [0, 0, -1, 0]

Q_EQ_RHS2 = 3
Q_EQ_COE2_Y = [0, -0.5, -2]
Q_EQ_COE2_Z = [0, 0, 0, -1]

# Add Variables
y = {}
z = {}

for i in range(3):
    y[i] = dual.addVar(name = 'y('+str(i)+')', lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY)
    if i == 0 or i == 2:
        y[i] = dual.addVar(name = 'y('+str(i)+')', lb = 0, ub = gp.GRB.INFINITY)
    
for i in range(4):
    z[i] = dual.addVar(name = 'z('+str(i)+')', lb = 0, ub = gp.GRB.INFINITY)
    if i >= 2:
        z[i] = dual.addVar(name = 'z('+str(i)+')', lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY)
    
# Set Objective

dual.setObjective(sum(Q_objcoe_Y[i] * y[i] for i in range(3)) + sum(Q_objcoe_Z[i] * z[i] for i in range(4)), gp.GRB.MAXIMIZE)

# Add Constraints

dual.addConstr(sum(Q_INEQ_COE1_Y[i] * y[i] for i in range(3)) + sum(Q_INEQ_COE1_Z[i] * z[i] for i in range(4)) >= Q_INEQ_RHS1, 'INEQ1')
                 
dual.addConstr(sum(Q_INEQ_COE2_Y[i] * y[i] for i in range(3)) + sum(Q_INEQ_COE2_Z[i] * z[i] for i in range(4)) >= Q_INEQ_RHS2, 'INEQ2')

dual.addConstr(sum(Q_EQ_COE1_Y[i] * y[i] for i in range(3)) + sum(Q_EQ_COE1_Z[i] * z[i] for i in range(4)) == Q_EQ_RHS1, 'EQ1')

dual.addConstr(sum(Q_EQ_COE2_Y[i] * y[i] for i in range(3)) + sum(Q_EQ_COE2_Z[i] * z[i] for i in range(4)) == Q_EQ_RHS2, 'EQ2')
                 
dual.addConstr(2 * z[0] * z[1] >= z[2] * z[2] + z[3] * z[3], 'CONE')

dual.update()

dual.setParam(gp.GRB.Param.Threads, 8)
dual.optimize()

Academic license - for non-commercial use only
Changed value of parameter Threads to 8
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 4 rows, 11 columns and 12 nonzeros
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [5e-01, 2e+00]
  QMatrix range    [1e+00, 2e+00]
  Objective range  [4e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 1 rows and 5 columns
Presolve time: 0.02s
Presolved: 5 rows, 8 columns, 16 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -6.24200231e+01  2.07094623e+01  2.87e+00 8.93e+00  2.46e+01     0s
   1  -3.18757423e+01 -1.57206538e+00  8.89e-01 8.78e-01  5.63e+00   

In [3]:
dual.write("file.lp")